In [32]:
import pandas as pd
import numpy as np

pd.set_option('max_rows', 20)

DAILY_DATA_FOLDER = 'D:/analytics/stock/hist'
DAILY_SAMPLE_PATH = 'D:/analytics/stock/hist/002415.csv'

df = pd.read_csv(DAILY_SAMPLE_PATH, index_col=0, parse_dates=True, \
                 usecols=['date', 'close', 'p_change', 'ma5', 'ma10', 'ma20'], \
                 error_bad_lines=False)
df

,close,p_change,ma5,ma10,ma20
date,,,,,
2016-04-22,31.17,2.37,30.686,30.857,30.809
2016-04-21,30.45,0.79,30.732,30.806,30.803
2016-04-20,30.21,-1.60,30.880,30.831,30.875
2016-04-19,30.70,-0.65,31.064,30.946,30.953
2016-04-18,30.90,-1.59,31.004,31.016,31.038
2016-04-15,31.40,0.67,31.028,30.976,31.037
2016-04-14,31.19,0.19,30.880,30.918,30.967
2016-04-13,31.13,2.40,30.782,30.909,30.868
2016-04-12,30.40,-2.00,30.828,30.781,30.792


In [5]:
r = df.resample('W')
# daily average price change percent on weekly basis
r['p_change'].mean().dropna().sort_values()

date
2015-07-12   -5.413333
2015-06-21   -3.862000
2013-12-08   -2.440000
2015-03-08   -2.382500
2014-03-23   -2.336000
                ...   
2015-04-05    2.442000
2013-06-16    2.645000
2014-12-07    2.842000
2015-01-11    3.418000
2015-05-24    6.086000
Name: p_change, dtype: float64

In [2]:
import pandas as pd
import numpy as np

WEEKLY_DATA_FOLDER = 'E:/analytics/stock/hist-W-2016-0627'
WEEKLY_SAMPLE_PATH = 'E:/analytics/stock/hist-W-2016-0627/002643.csv'

df = pd.read_csv(WEEKLY_SAMPLE_PATH, index_col=0, parse_dates=True, \
                 usecols=['date', 'close', 'p_change', 'ma5', 'ma10', 'ma20'], \
                 error_bad_lines=False)
df

,close,p_change,ma5,ma10,ma20
date,,,,,
2016-06-27,58.10,3.11,52.744,50.356,42.310
2016-06-24,56.35,7.33,50.660,48.755,41.205
2016-06-17,52.50,6.56,49.130,47.041,40.087
2016-06-08,49.27,3.73,48.250,45.011,39.300
2016-06-03,47.50,-0.38,47.926,43.202,38.561
2016-05-27,47.68,-2.09,47.968,41.792,38.409
2016-05-20,48.70,1.25,46.850,40.089,38.051
2016-05-13,48.10,0.94,44.952,38.317,37.742
2016-05-06,47.65,-0.13,41.772,36.678,37.301


In [3]:
r_df = df.sort_index()
r_df.rolling(window=5)['close'].mean()

date
2016-01-08       NaN
2016-01-15       NaN
2016-01-22       NaN
2016-01-29       NaN
2016-02-05    37.140
2016-02-19    35.772
2016-02-26    35.592
2016-03-04    34.584
2016-03-11    33.980
2016-03-18    32.910
2016-03-25    32.068
2016-04-01    31.584
2016-04-08    31.682
2016-04-15    33.328
2016-04-22    35.616
2016-04-29    38.478
2016-05-06    41.772
2016-05-13    44.952
2016-05-20    46.850
2016-05-27    47.968
2016-06-03    47.926
2016-06-08    48.250
2016-06-17    49.130
2016-06-24    50.660
2016-06-27    52.744
Name: close, dtype: float64

In [18]:
s = df['ma5']
change_point = None
for i in range(0, len(s)-1):
    if s[i] < s[i+1]:
        change_point = i
        break
if not change_point:
    change_point = len(s) - 1
startdate = df.index[change_point]
increase = round(s[0] / s[change_point] - 1, 4) * 100
period = s[0:change_point+1]
print(startdate, increase, change_point+1, df[:change_point+1]['p_change'].mean(), \
      round(df[:change_point+1]['p_change'].std(), 2))

2016-06-03 00:00:00 10.05 5 4.07 3.07


In [1]:
import re
import glob
import os
import os.path
import pandas as pd
import sys

WEEKLY_DATA_FOLDER = 'E:/analytics/stock/hist-W-2016-0627'

reg = re.compile(r'(\d{6}).csv')
stocks = {t[1].group(1):t[0] for t in ((x, reg.search(x)) for x in glob.glob(WEEKLY_DATA_FOLDER + '/*.csv')) if t[1]}
# [os.path.isfile(x) for x in list(stocks.values())[:5]]

def find_increase_trend(path):
    df = pd.read_csv(path, index_col=0, parse_dates=True, \
                usecols=['date', 'close', 'p_change', 'ma5', 'ma10', 'ma20'], \
                error_bad_lines=False)
    if len(df) == 0:
        return None
    
    s = df['ma5']
    # the position from which MA starts to rise
    change_point = None
    for i in range(0, len(s)-1):
        if s[i] < s[i+1]:
            change_point = i
            break
    if change_point is None:
        change_point = len(s) - 1
    # if there is no increase in most recent period
    if change_point == 0:
        return None
    else:
        startdate = df.index[change_point]
        # use close price of two ends to calc total increase percent
        total_increase = round(df.iat[0, 0] / df.iat[change_point, 0] - 1, 4) * 100
        # the observation at change point doesn't count as increase, but only later ones
        trend_length = change_point
        # mean week-over-week increase percent over this period
        mean_increase = round(df[:change_point]['p_change'].mean(), 2)
        first_above_mean_position = df.index.get_loc(df[df['p_change'] >= mean_increase].index[0])
        # num of recent consecutive obserations whose WoW increase is lower than mean
        recent_below_mean_count = first_above_mean_position
        std = round(df[:change_point]['p_change'].std(), 2)
        return startdate, total_increase, trend_length, mean_increase, std, recent_below_mean_count

resultmap = {}
for code in stocks:
    try:
#         print('processing ', code)
        res = find_increase_trend(stocks[code])
        ''' take as valid entry when following conditions are met:
                1. increasing trend lasts longer than 3 observations 
                2. actual increase percent over the period is above 10% (MA trails behind actual varation)
                3. increasing trend didn't considerably slowdown lately
        '''
        if res and res[2] >= 3 and res[1] > 10 and res[-1] <= 1:
            resultmap[code] = res
    except Exception as ex:
        print('error occurred in processing %s: %s' % (code, ex))
df = pd.DataFrame.from_dict(resultmap, orient='index')
df.columns = ['startdate', 'increase', 'length', 'mean', 'std', 'recent_slowdown_length']
df

,startdate,increase,length,mean,std,recent_slowdown_length
601222,2016-05-27,17.46,5,3.38,5.40,1
000585,2016-06-08,10.40,3,3.65,9.83,1
002749,2016-05-27,19.51,5,3.74,5.43,1
002609,2016-06-03,14.87,4,3.55,2.64,1
002097,2016-05-20,16.94,6,2.75,5.07,0
002448,2016-05-20,12.80,6,2.10,4.23,0
002165,2016-05-06,15.16,8,1.84,3.85,0
002635,2016-05-20,10.53,6,1.76,4.33,0
002309,2016-03-25,23.74,14,1.62,4.33,0
002217,2016-03-25,23.03,14,1.58,4.39,0


In [35]:
OUTPUT_PATH = 'D:/analytics/stock/analysis_weekly.csv'
df.to_csv(OUTPUT_PATH)

In [2]:
find_increase_trend(WEEKLY_DATA_FOLDER + '/600529.csv')

(Timestamp('2016-06-17 00:00:00'), -0.12, 2, -0.04, 2.61, 0)

In [22]:
import numpy as np

BASIC_DATA_PATH = r'E:\analytics\stock\basic.csv'
basic_df = pd.read_csv(BASIC_DATA_PATH, index_col=False, parse_dates=['timeToMarket'], dtype={'code':np.str}, \
                       usecols=['code', 'pe', 'pb', 'outstanding', 'totals', 'esp', 'timeToMarket'], \
                       error_bad_lines=False, infer_datetime_format=True)
basic_df

,code,pe,outstanding,totals,esp,pb,timeToMarket
0,603958,21.30,5436.00,21736.0,0.155,3.19,20160629
1,002802,30.34,2700.00,10800.0,0.113,3.78,20160629
2,600069,0.00,82537.40,124910.29,-0.035,3.10,19970430
3,002325,34.06,91356.09,120185.09,0.068,3.34,20091222
4,002651,100.32,31280.41,100250.0,0.025,5.29,20120106
5,300074,0.00,43451.62,52800.0,-0.017,5.79,20100427
6,002666,52.27,44626.83,75432.92,0.045,2.66,20120327
7,600767,0.00,34091.01,34101.01,-0.052,18.54,19961115
8,600528,65.12,145920.00,145920.0,0.043,2.63,20010528
9,000821,2233.90,34521.94,47773.26,0.002,5.00,19980626


In [50]:
REPORT_DATA_FOLDER = r'E:/analytics/stock/report'

reg = re.compile(r'(\d{4}-\d).csv')
reports = {t[1].group(1):t[0] for t in ((x, reg.search(x)) for x in glob.glob(REPORT_DATA_FOLDER + '/*.csv')) if t[1]}
report_dfs = []
report_terms = []
for term in reports:
    rdf = pd.read_csv(reports[term], index_col=False, dtype={'code':np.str}, \
                           usecols=['code', 'roe', 'profits_yoy'], \
                           error_bad_lines=False)
    rdf.index = rdf['code']
    del rdf['code']
    print(term, ':')
    print(rdf.head(5))
    report_dfs.append(rdf)
    report_terms.append(term)
print(report_dfs)
all_report_df = pd.concat([report_dfs[0], report_dfs[1]], axis=1, join='inner')

2016-1 :
          roe  profits_yoy
code                      
002801   4.53        22.94
603909   4.16          NaN
300518   3.65        23.94
300519  12.11         3.35
300515   2.48      -425.65
2015-4 :
          roe  profits_yoy
code                      
002801  22.17        16.01
300519  44.38        12.43
603909  19.34         7.98
300518  20.97         5.01
601127  15.67        23.93
2015-3 :
          roe  profits_yoy
code                      
603959   9.32          NaN
603701  17.07          NaN
002791  11.51          NaN
300484  18.75          NaN
002792  19.76          NaN
2015-2 :
          roe  profits_yoy
code                      
603608   7.11          NaN
300500   8.45          NaN
300499   5.12          NaN
603377  26.96          NaN
601900   8.73          NaN
[          roe  profits_yoy
code                      
002801   4.53        22.94
603909   4.16          NaN
300518   3.65        23.94
300519  12.11         3.35
300515   2.48      -425.65
603339   3.85     

ValueError: Shape of passed values is (4, 4282), indices imply (4, 2514)